# Tugas Tokenisasi abstrak

# **Ambil Data PTA Informatika**

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re, sys, time

In [2]:
BASE_URL = "https://pta.trunojoyo.ac.id/c_search/byprod"
def get_max_page(prodi_id):
    url = f"{BASE_URL}/{prodi_id}/1"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")

    # Cari tombol >> (last page)
    last_page = soup.select_one('ol.pagination a:contains("»")')
    if last_page and "href" in last_page.attrs:
        href = last_page["href"]
        # Pecah URL -> ambil angka terakhir
        max_page = int(href.split("/")[-1])
        return max_page

    # fallback kalau pagination tidak ada
    return 1
print(get_max_page(10))

172


/home/codespace/.local/lib/python3.12/site-packages/soupsieve/css_parser.py:876: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


In [3]:
def print_progress(prodi_id, prodi, current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 20
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\r[{prodi_id}] {prodi} - Page {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()
    if current_page == total_pages:
        sys.stdout.write('\n')

In [4]:
import time, re, sys, requests, pandas as pd
from bs4 import BeautifulSoup

BASE_URL = "https://pta.trunojoyo.ac.id/c_search/byprod"  # tetap

def pta_ti_only():
    start_time = time.time()

    data = {
        "id": [],
        "judul": [],
        "abstrak_id": [],
        "prodi": []
    }

    # Teknik Informatika = 10 di situs (byprod/10/halaman)
    total_prodi = 10  # tetap pakai 1..10, tapi difilter nama prodi
    total_pages = 0
    max_pages_dict = {}

    for i in range(1, total_prodi + 1):
        max_page = get_max_page(i)
        max_pages_dict[i] = max_page
        total_pages += max_page

    for i in range(1, total_prodi + 1):
        max_page = max_pages_dict[i]

        for j in range(1, max_page + 1):
            url = f"{BASE_URL}/{i}/{j}"
            r = requests.get(url, timeout=30)
            soup = BeautifulSoup(r.content, "html.parser")

            isii = soup.select_one('div#begin')
            if not isii:
                continue
            prodi_full = isii.select_one('h2').text.strip()
            prodi = prodi_full.replace("Journal Jurusan ", "")

            if prodi.lower() != "teknik informatika":
                continue  # hanya TI

            jurnals = soup.select('li[data-cat="#luxury"]')

            for jurnal in jurnals:
                a_tag = jurnal.select_one('a.gray.button')
                if not a_tag or not a_tag.has_attr('href'):
                    continue
                link_keluar = a_tag['href']

                id_match = re.search(r"/detail/(\d+)", link_keluar)
                pta_id = id_match.group(1) if id_match else None

                response = requests.get(link_keluar, timeout=30)
                soup1 = BeautifulSoup(response.content, "html.parser")
                isi = soup1.select_one('div#content_journal')
                if not isi:
                    continue

                t = isi.select_one('a.title')
                judul = t.text.strip() if t else "N/A"

                # ambil abstrak Indonesia saja sesuai kebutuhan
                paragraf = isi.select('p[align="justify"]')
                abstrak_id = paragraf[0].get_text(strip=True) if len(paragraf) > 0 else "N/A"

                data["id"].append(pta_id)
                data["judul"].append(judul)
                data["abstrak_id"].append(abstrak_id)
                data["prodi"].append(prodi)

            print_progress(i, prodi, j, max_page)

        sys.stdout.write("\n")

    df = pd.DataFrame(data)
    # simpan pakai nama yang diminta
    df.to_csv("pta_informatika.csv", index=False, encoding="utf-8-sig")

    elapsed = int(time.time() - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    print("\n✅ Data Teknik Informatika berhasil dikumpulkan!")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    # tampilkan 5 baris saja
    print("\nPreview 5 data teratas:")
    print(df.head(5))

    return df

# jalankan
pta_ti_only()











[10] Teknik Informatika - Page 172/172 [████████████████████] 100.00%


✅ Data Teknik Informatika berhasil dikumpulkan!
📊 Total entri: 858
⏱️ Waktu eksekusi: 0 jam 27 menit 35 detik

Preview 5 data teratas:
             id                                              judul  \
0  040411100468  PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...   
1  040411100476  APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...   
2  040411100480  RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\r\n...   
3  070411100070  SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...   
4  080411100115  SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...   

                                          abstrak_id               prodi  
0  Sistem  informasi  akademik  (SIAKAD) merupaka...  Teknik Informatika  
1  Berjalannya koneksi jaringan komputer dengan l...  Teknik Informatika  
2  Web server adalah sebuah perangkat lunak serve...  Teknik Informatika  
3  Penjadwalan  kuliah  di  Perguruan  Tinggi  me...  Teknik In

,id,judul,abstrak_id,prodi
0,040411100468,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...,Sistem informasi akademik (SIAKAD) merupaka...,Teknik Informatika
1,040411100476,APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,Berjalannya koneksi jaringan komputer dengan l...,Teknik Informatika
2,040411100480,RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\r\n...,Web server adalah sebuah perangkat lunak serve...,Teknik Informatika
3,070411100070,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,Penjadwalan kuliah di Perguruan Tinggi me...,Teknik Informatika
4,080411100115,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,Seiring perkembangan teknologi yang ada diduni...,Teknik Informatika
...,...,...,...,...
853,160411100032,PENERAPAN ALGORITMA LONG-SHORT TERM MEMORY UNT...,Investasi saham selama ini memiliki resiko ker...,Teknik Informatika
854,160411100182,SISTEM PENCARIAN TEKS AL-QURAN TERJEMAHAN BERB...,Information Retrieval (IR) merupakan pengambil...,Teknik Informatika
855,160411100077,KLASIFIKASI KOMPLEKSITAS VISUAL CITRA SAMPAH M...,Klasifikasi citra merupakan proses pengelompok...,Teknik Informatika
856,160411100084,IDENTIFIKASI BINER ATRIBUT PEJALAN KAKI MENGGU...,Identifikasi atribut pejalan kaki merupakan sa...,Teknik Informatika


# **Tokenisasi**

In [15]:
import pandas as pd
import re

# 1. Baca data hasil scraping
df = pd.read_csv("pta_informatika.csv")

# 2. Gabungkan semua abstrak jadi satu teks panjang
teks = " ".join(df["abstrak_id"].dropna().tolist())
print("=== Preview Teks Abstrak Gabungan ===")
print(teks[:500])  # tampilkan 500 karakter pertama

# 3. Tokenisasi manual pakai regex (ambil kata saja)
tokens = re.findall(r'\b\w+\b', teks.lower())

# 4. Print preview 5 token pertama
print("\n=== Preview Hasil Tokenisasi (5 kata pertama) ===")
print(tokens[:25])

print("\n✅ Tokenisasi selesai, total kata:", len(tokens))


=== Preview Teks Abstrak Gabungan ===
Sistem  informasi  akademik  (SIAKAD) merupakan  sistem  informasi  yang  berfungsi  menangani pengelolaan  dan  penyajian  data-data  akademik,  yang  oleh pihak  fakultas  SIAKAD  dianggap  sangat  penting  dalam memberikan  pelayanan  mahasiswa  yang  membutuhkan informasi akademik. Di Universitas Trunojoyo telah tersedia SIAKAD,  namun  masih  menggunakan  database  terpusat. Sistem seperti ini memberikan kelebihan yaitu perawatannya mudah  selain  itu  juga  membutuhkan  sedikit  biaya,  na

=== Preview Hasil Tokenisasi (5 kata pertama) ===
['sistem', 'informasi', 'akademik', 'siakad', 'merupakan', 'sistem', 'informasi', 'yang', 'berfungsi', 'menangani', 'pengelolaan', 'dan', 'penyajian', 'data', 'data', 'akademik', 'yang', 'oleh', 'pihak', 'fakultas', 'siakad', 'dianggap', 'sangat', 'penting', 'dalam']

✅ Tokenisasi selesai, total kata: 144883


# **Stopword Removal**

In [11]:
pip install Sastrawi pyspellchecker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pyspellchecker]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from spellchecker import SpellChecker
from collections import Counter

In [ ]:
# 4. Stopword Removal
stop_factory = StopWordRemoverFactory()
stopwords = set(stop_factory.get_stop_words())
tokens_no_stop = [t for t in tokens if t not in stopwords]
print("\n=== Setelah Stopword Removal (25 kata pertama) ===")
print(tokens_no_stop[:25])


=== Setelah Stopword Removal (5 kata pertama) ===
['sistem', 'informasi', 'akademik', 'siakad', 'merupakan', 'sistem', 'informasi', 'berfungsi', 'menangani', 'pengelolaan', 'penyajian', 'data', 'data', 'akademik', 'pihak', 'fakultas', 'siakad', 'dianggap', 'sangat', 'penting', 'memberikan', 'pelayanan', 'mahasiswa', 'membutuhkan', 'informasi']


# **Stemming**

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
tokens_stemmed = [stemmer.stem(t) for t in tokens_no_stop]
print("\n=== Setelah Stemming (25 kata pertama) ===")
print(tokens_stemmed[:25])



=== Setelah Stemming (5 kata pertama) ===
['sistem', 'informasi', 'akademik', 'siakad', 'rupa', 'sistem', 'informasi', 'fungsi', 'tangan', 'kelola', 'saji', 'data', 'data', 'akademik', 'pihak', 'fakultas', 'siakad', 'anggap', 'sangat', 'penting', 'beri', 'layan', 'mahasiswa', 'butuh', 'informasi']


# **Cek Ejaan**

In [14]:
# 1) Sumber token: pilih salah satu yang konsisten
#    Rekomendasi: gunakan hasil stemming agar konsisten bentuk katanya
#    Jika belum ada di kernel saat ini, bisa rebuild dari CSV
import pandas as pd
import re

try:
    tokens_source = tokensstemmed  # sudah dibuat di sel stemming sebelumnya
except NameError:
    # fallback: rebuild dari CSV
    df = pd.read_csv("pta_informatika.csv")
    teks = " ".join([s.strip() for s in df["abstrak_id"].dropna().astype(str) if s.strip()]).lower()
    tokens_source = re.findall(r"[a-zA-Záéíóúàèìòùâêîôûäëïöüñ\-]+", teks)  # regex kata
    # optional: lakukan stopword + stemming lagi jika perlu

# 2) Inisialisasi spell checker
from spellchecker import SpellChecker
spell = SpellChecker(language=None)  # no default language

# 3) Bangun kamus dasar dari korpus sendiri (unsupervised):
#    - tambahkan semua token unik sebagai known words agar nama teknis tidak dianggap salah
#    - tetap biarkan spellchecker mendeteksi token aneh/typo langka
vocab = set(tokens_source)
spell.word_frequency.load_words(vocab)

# 4) Temukan kata yang dianggap salah eja
misspelled = spell.unknown(tokens_source)

# 5) Usulkan koreksi (heuristik sederhana)
suggestions = {}
for word in list(misspelled)[:1000]:  # batasi agar cepat; hapus batas jika ingin lengkap
    correction = spell.correction(word)
    if correction and correction != word:
        suggestions[word] = correction

print("=== Contoh 20 kata salah eja & saran ===")
for i, (w, c) in enumerate(list(suggestions.items())[:20], start=1):
    print(f"{i}. {w} -> {c}")

# 6) Terapkan koreksi ringan: hanya ganti jika confidence relatif masuk akal
def correct_token(t):
    if t in suggestions:
        return suggestions[t]
    return t

tokens_checked = [correct_token(t) for t in tokens_source]

print(f"Total token: {len(tokens_source)} | Salah eja terdeteksi: {len(misspelled)} | Setelah koreksi: {len(tokens_checked)}")


=== Contoh 20 kata salah eja & saran ===
Total token: 139450 | Salah eja terdeteksi: 0 | Setelah koreksi: 139450


# **Hitung Frekuensi Kata**

In [16]:
# 0) Jika belum ada tokens, rebuild cepat dari CSV
import re, pandas as pd
try:
    tokens
except NameError:
    df = pd.read_csv("pta_informatika.csv")
    teks = " ".join([s.strip() for s in df["abstrak_id"].dropna().astype(str) if s.strip()]).lower()
    tokens = re.findall(r"[a-zA-Záéíóúàèìòùâêîôûäëïöüñ\-]+", teks)  # token sederhana huruf & hyphen

# 1) Hitung frekuensi
from collections import Counter
freq = Counter(tokens)  # atau Counter(tokens_checked) jika pakai hasil koreksi ejaan

# 2) Urutkan dari yang terbanyak
freq_sorted = pd.DataFrame(freq.most_common(), columns=["kata", "frekuensi"])

# 3) Simpan ke CSV (UTF-8 BOM)
freq_sorted.to_csv("frekuensi_kata_informatika.csv", index=False, encoding="utf-8-sig")

# 4) Preview 20 teratas
print("=== 20 kata paling sering ===")
print(freq_sorted.head(20))


=== 20 kata paling sering ===
           kata  frekuensi
0          yang       5580
1           dan       3497
2        dengan       2962
3         untuk       2327
4           ini       2171
5         dalam       1930
6          pada       1869
7          dari       1713
8        sistem       1570
9        metode       1401
10  menggunakan       1350
11        dapat       1262
12        hasil       1170
13         data       1086
14           di        955
15       adalah        954
16   penelitian        867
17        nilai        807
18       proses        799
19    digunakan        746
